# Text 2: Latent semantic indexing
**Internet Analytics - Lab 4**

---

**Group:** K

**Names:**

* Kim Lan Phan Hoang
* Robin Lang

---

#### Instructions

*This is a template for part 2 of the lab. Clearly write your answers, comments and interpretations in Markodown cells. Don't forget that you can add $\LaTeX$ equations in these cells. Feel free to add or remove any cell.*

*Please properly comment your code. Code readability will be considered for grading. To avoid long cells of codes in the notebook, you can also embed long python functions and classes in a separate module. Don’t forget to hand in your module if that is the case. In multiple exercises, you are required to come up with your own method to solve various problems. Be creative and clearly motivate and explain your methods. Creativity and clarity will be considered for grading.*

In [ ]:
import pickle
import numpy as np
from scipy.sparse.linalg import svds

## Exercise 4.4: Latent semantic indexing

### Apply SVD with K = 300 to your term-document matrix X from the previous exercise.

### Describe the rows and columns of U and V, and the values of S.

### Print the top-20 eigenvalues of X

## Exercise 4.5: Topic extraction

### Extract the topics from the term-document matrix X using the low-rank approximation.

### Print the top-10 topics as a combination of terms and a combination of documents.

### Give a label to each of them.

## Exercise 4.6: Document similarity search in concept-space

### Implement a search function using LSI concept-space.

### Search for "facebook" in both VSM and LSI. How does it compare? Why?

### Search for "markov chains" and compare with the previous section.

## Exercise 4.7: Document-document similarity

### Find the classes that are the most similar to Internet Analytics.

### Write down the equation to efficiently compute the similarity between documents.

### Print the top 5 classes most similar to COM-308.